## Create database

In [1]:
from pymilvus import connections, db

conn = connections.connect(host='10.37.239.102', port=19530)

In [ ]:
database = db.create_database("face_recognition")

db.using_database("face_recognition")

In [2]:
db.list_database()

['default', 'face_recognition']

## Create collection

In [8]:
from pymilvus import CollectionSchema, FieldSchema, DataType

face_id = FieldSchema(name="face_id", description="id", dtype=DataType.INT64, is_primary=True, auto_id=False)
face_vector = FieldSchema(name="face_vector", description="face", dtype=DataType.FLOAT_VECTOR, dim=512)
schema = CollectionSchema(fields=[face_id, face_vector], description="face recognition", enable_dynamic_field=True)
collection_name = "face_recognition"

In [9]:
from pymilvus import Collection
collection = Collection(
    name=collection_name,
    schema=schema,
    )

In [10]:
from pymilvus import utility
utility.list_collections()

['hello_milvus', 'face_recognition']

## Insert entities

In [ ]:
import numpy as np

arr_id = []
arr_face = []
for i in range(29):
    data = np.load("emb_db/{}.npy".format(i))
    for d in data:
        arr_id.append(i)
        arr_face.append(d)
entities = [
    [i for i in arr_id],
    [d.tolist() for d in arr_face]
]

collection.insert(entities)
collection.flush()

## Builds indexes

In [ ]:
index_params = {
    "metric_type":"L2", 
    "index_type":"IVF_FLAT",
    "params":{"nlist":512}
}

collection.create_index(
    field_name="face_vector",
    index_params=index_params
)

utility.index_building_progress("face_recognition")

## Search

In [11]:
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

NameError: name 'entities' is not defined

In [12]:
collection.load()
res = collection.query(
    expr="face_id in [2]",
    offset=0,
    limit=3,
    output_fields=["face_id", "face_vector"]
)
print(res)

RPC error: [load_collection], <MilvusException: (code=700, message=collection=face_recognition: index not found)>, <Time:{'RPC start': '2023-12-21 15:02:53.415042', 'RPC error': '2023-12-21 15:02:53.485040'}>


MilvusException: <MilvusException: (code=700, message=collection=face_recognition: index not found)>

In [6]:
collection.load()
collection.search(vectors_to_search, "face_vector", search_params, limit=1, output_fields=["face_id"])

NameError: name 'collection' is not defined

## Drop database

In [ ]:
from pymilvus import utility, db

db.using_database("face_recognition")
utility.drop_collection("face_recognition")
db.drop_database("face_recognition")

db.list_database()